Queen's Campus Campus Pathfinding
Reinforcment Learning agent who uses Q-Learning to find the shortest path between two points on Campus

CISC 474
names and SN's (insert before handing in - Not while on Github though)
- 
- 
- 
-

Reference Materials:

[Overpass API/Overpass QL](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL): API and Query Language for accessing Open Street Maps data.

[Overpass Turbo](https://overpass-turbo.eu/): Website to help build and visualize Overpass API calls.

[OverPy](https://python-overpy.readthedocs.io/en/latest/api.html): Python Wrapper for Overpass

In [1]:
###
## TO DO

# get_ways_fromNode(node,ways)
# get_node_fromWay(node,way)
# get_time(node_a, node_b, way)

# World.print_world(self)
#Wokring on it (OScar)

# Look over world and agent function logic. Nodes are objects and if we want to 
#   specifically refer to them perhaps we should do it as node.id (For example for building dictionaries)

#Do we need values? ie enviroment.set next values if not then delete it
####

In [2]:
!pip install OSMPythonTools
!pip install overpy
from logging import exception
!pip install -e "git+https://github.com/marceloprates/prettymaps#egg=prettymaps"

!npm cache clean -f
!npm install -g n
!n stable
!npm install -g npm
!npm install -g osmtogeojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining prettymaps from git+https://github.com/marceloprates/prettymaps#egg=prettymaps
  Updating ./src/prettymaps clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 3f3b8be923a9c0abda10fbc6f002ceb048c8e7f2
  Attempting uninstall: prettymaps
    Found existing installation: prettymaps 0.1.3
    Can't uninstall 'prettymaps'. No files were found to uninstall.
  Running setup.py develop for prettymaps
node: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by node)
node: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by node)
     copying : node/18.12.1
node: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' no

In [3]:
#Not really using this at the moment, but might have to, to do queries between nodes
# Api imports and set up to: import and access OpenStreetMaps
from OSMPythonTools.api import Api
from OSMPythonTools.overpass import Overpass

OSMapi = Api()
queens = OSMapi.query('way/12030883') 
print(queens)
overpass = Overpass()

In [4]:
# Main python wrapper for acessing OSM tools

import overpy
api = overpy.Overpass()

## Bounding box - Limits the scope of our api 
## Campus is (roughly)  ==  [bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918]
# Import all nodes and ways (edges) locally

all_campus_result = api.query('[bbox: 44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(way[highway=footway];way[highway=pedestrian];way[highway=steps];way[highway=corridor];way[highway=path];way[highway=service];);(._;>;);out;')
#This holds ways and nodes

#Nodes
print("All nodes on campus: "+str(len(all_campus_result.nodes)))
print(all_campus_result.nodes[1])

#Ways
print("All ways on campus: "+str(len(all_campus_result.ways)))
print(all_campus_result.ways[1])

#Relations - There are none non campus, we could build them manually if needed tho  
#all_campus_result.append((api.query('[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(relation;);out;')).relations)
try:
  queens_campus_relations = api.query('[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(relation;);out;')
  queens_campus_relations.relations[0]
except:
  print("No relations on campus")
else:
  print("All Relations on campus: "+str(len(all_campus_result.relations)))
  print(queens_campus_relations.relations[0])


All nodes on campus: 2362
<overpy.Node id=27495897 lat=44.2279960 lon=-76.4908268>
All ways on campus: 722
<overpy.Way id=4964337 nodes=[30342171, 7434257090, 7434257091, 32871496, 32871497, 305089193, 32871498, 7434257092, 32871499]>
All Relations on campus: 0
<overpy.Relation id=36847>


In [5]:
## Sample functions

# Get all nodes in a way
def get_node_fromWay(node, way):
    pass
    
# Calulate the time walking a "Way"
def get_time(node_a, node_b, way):
    pass

# maybe get time from

# Search for a tag in an area "query" This could be used for input/output coordinates/location

#[out:json][timeout:250];
#(
#  area[name="Palo Alto"];
#  way(area)["building"];
#  relation(area)["building"];
#);
#out body;
#>;
#out skel qt;
def get_nodes_with_tag(tag, nodes):
    n_list = []
    for n in nodes:
        if tag in vars(n)['tags'].keys():
            n_list.append(n)
    return n_list

# returns name of building
def _node_to_str(node):
    return vars(node)['tags']['name']

# returns name of node (if it exists)
def _str_to_node(string, nodes):
    for n in nodes:
        if string in vars(n)['tags']['name']:
            return n
    return"no node with this name"

# Get random node in set
def random_node(set):
    pass

#Could be used for input/output coordinates/location
def get_closest_node_fromCoordinate(latitude, longitude):
  # figure out how to input coordinate and in what order they are usually shown
  pass
  #Output node
    

In [6]:
from copy import deepcopy
import numpy as np

class World:
    
    def __init__(self, allData:overpy.Result, start, terminal): 
    
        # nodes (list) = the nodes
        # ways (list) = the ways
        # start (node) = the starting node
        # terminal (node) = the goal node

        # policy (dict) = each node a: {each potential goal node: best first way to take to reach this goal}
        # values (dict) = each node a: {each potential goal node: {each way from node a: value}}

        self._allData = allData

        self._nodes = allData.nodes
        self._ways = allData.ways
        self._start = start ### What type is this coordinates or node? Figure out after making interface
        self._terminal = terminal
        
        self._policy = {}
        self._values = {}
        
        self._values_next = {}
        self._values_next = self.create_next_values() ###do we need this?

        self._currentNode = start
        self.isEnd = False
        
        for node in self._nodes:
            self._values[node] = {}
            self._values[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            self._policy[node] = {}
            self._policy[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            for goal_node in self._values[node]:
                self._values[node][goal_node] = {}
                self._values[node][goal_node] = dict(zip(get_ways_fromNode(node,self._ways), [None]*len(get_ways_fromNode(node,self._ways))))
                self._policy[node][goal_node] = random.choice(self._values[node][goal_node].keys())
                for possible_way in self._values[node][goal_node]:
                    self._values[node][goal_node][possible_way] = 0
    
    def reset(self):
        for node in self._nodes:
            self._values[node] = {}
            self._values[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            self._policy[node] = {}
            self._policy[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            for goal_node in self._values[node]:
                self._values[node][goal_node] = {}
                self._values[node][goal_node] = dict(zip(get_ways_fromNode(node,self._ways), [None]*len(get_ways_fromNode(node,self._ways))))
                self._policy[node][goal_node] = random.choice(self._values[node][goal_node].keys())
                for possible_way in self._values[node][goal_node]:
                    self._values[node][goal_node][possible_way] = 0
        self.create_next_values()
        
    # you get the reward when you take the way
    # a reward of 0 when you reach the terminal state
    # a negative reward when you take an way
    def _get_reward(self, node_a, node_b, way):
        if node_b == self._terminal:
            return 0
        else:
            return -get_time(node_a,node_b,way)
    
    def _get_next_node(self, node, way):
        return get_node_fromWay(node, way)
    
    # returns all nodes in the world
    def get_nodes(self):
        return self._nodes

    # Returns all node id from input list of nodes (Ways or Nodes)
    def get_element_ids(elementList):
      idList = []
      for element in elementList:
        idList.append(element.id)
      return idList

    
    # returns each way from this node to terminal ### we dont know this?
    def get_ways_toTerminal(self, from_node:overpy.Node):
        return self._values[from_node][gw._terminal].keys()
    
    # returns the value of each way from this node to terminal
    def get_values(self, node):
        return self._values[node][gw._terminal]
    
    # values should be a dictionary of {(goal node: (possible ways: value)),}
    def set_values(self, node, values): ###
        self._values_next[node][gw._terminal] = values
        
    def set_next_values(self): ###
        self._values = deepcopy(self._values_next)
        
    def create_next_values(self): ###
        for node in self._nodes:
            self._values_next[node] = {}
            self._values_next[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            self._policy[node] = {}
            self._policy[node] = dict(zip(self._nodes, [None]*len(self._nodes)))
            for goal_node in self._values[node]:
                self._values_next[node][goal_node] = {}
                self._values_next[node][goal_node] = dict(zip(get_ways_fromNode(node, self._ways), [None]*len(get_ways_fromNode(node, self._ways))))
                self._policy[node][goal_node] = random.choice(self._values_next[node][goal_node].keys())
                for possible_way in self._values[node][goal_node]:
                    self._values_next[node][goal_node][possible_way] = 0
                    

    def get_ways_fromNode(self, node:overpy.Node):
      ## Gets all ways to move from a node
      # Input: Node to be considered
      # Output: list of ways
      way_list = []
      for each_way in self._allData.get_ways():
        idList = []

        for eachNode in each_way.nodes:
          idList.append(eachNode.id)
        if node.id in idList:
          way_list.append(each_way)

      if len(way_list) < 1:
        # Use this to delete any nodes not connected to the system, delete later
        raise exception("No ways connected to node")

      return way_list

    def isEndFunc(self): # Checks if the current state is at the goal and flags that the game has ended
      if (self.currentNode == self._terminal):
        self.isEnd = True
    
    def print_world(self):
        pass
        

In [7]:
def qlearning(world:World, alpha, gamma, e, max_eps=500):

    world.reset()
    q = {}

    for node in world.get_nodes():
        q[node] = {}
        if node != world._terminal:
            for way in get_ways_fromNode(node, world._ways):
                q[node][way] = 1
        elif node == world._terminal:
            for way in world.ways:
                q[node][way] = 0
                
    world.create_next_values()
    
    for i in range(0, max_eps):
        n = world._start
        
        while n != world._terminal:   
            rc = np.random.choice(['exploit','explore'], p=[1-e, e])
            if rc == 'exploit':
                v = np.argmax(list(q[n].values()))
                w = list(g[n][v])
            elif rc == "explore":
                w = np.random.choice(list(q[n]))
            n_prime = world._get_next_node(n,w)
            r = world._get_reward(n, n_prime, way)
            q[n][w] += alpha*(r + (gamma * np.max(list(q[n_prime].values())) - q[n][w]))
            values = {}
            for way in get_ways_fromNode(n, world._ways):
                values[way] = q[n][way]
            world.set_values(n, values)
            n = n_prime


        world.set_next_values()
        
    world.create_next_values()

In [8]:
nodes = all_campus_result.nodes
ways = all_campus_result.ways

# making a list of campus buildings from ways with the building tag
buildings = []
for w in ways:
    if 'building' in vars(w)['tags'].keys():
        if 'name' in vars(w)['tags'].keys():   
            buildings.append(w)
##print("node to string: ", _node_to_str(buildings[5]) )
#print("string to node: ", _str_to_node("Ontario Hall", buildings))

start = np.random.choice(nodes)
terminal = np.random.choice(nodes)

# world = World(nodes, ways, start, terminal)
alpha = 0.1
gamma = 0.9
e = 0.1

# qlearning(world, alpha=alpha, gamma=gamma, e=e)
# world.print_world()

Interface

In [9]:

# making a list of campus buildings from ways with the building tag
tempData = api.query('[bbox:44.2224495448,-76.5017059541,44.2298231916,-76.4903552918];(way;);out;')
buildingNames = []
wayId = []
for w in tempData.ways:
  #print(w.tags.keys())#[access'])
  if "building" in w.tags.keys():
    if "name" in w.tags.keys():
      buildingNames.append(w.tags["name"])
      wayId.append(w.id)

print(buildingNames)
    
    #if 'building' in vars(w)['tags'].keys():
    #    if 'name' in vars(w)['tags'].keys():  
    #        print(w) 
    #        buildings.append(w)
#print("node to string: ", _node_to_str(buildings[5]) )
#print("string to node: ", _str_to_node("Ontario Hall", buildings))

# Please uncomment a line to Choose a starting point
#s =         #Douglas Library
#s =          # Stauffer Library
s = all_campus_result.get_node(9224895888,True)         #Ontario Hall

# Please uncomment a line to Choose a starting point
#t =         #Douglas Library
#t =          # Stauffer Library
#t = all_campus_result.get_node(9224895888)         #Ontario Hall
t = all_campus_result.get_node(9130940237,True) #Ellis Hall

# Alternativly input coordinates to choose a starting point and ending point
#start_latitude = 
#start_longitude =
#s = coor_resolve_to_node(start_latitude,start_longitude) 

#end_latitude = 
#end_longitude =
#t = coor_resolve_to_node(end_latitude,end_longitude)  


def coor_resolve_to_node(latitude,longitide) -> overpy.Node:
  # Api call to get close nodes,
  #get their id
  #then find it in our node list and return
  pass


['Douglas Library', 'Stauffer Library', 'Robert Sutherland Hall', 'Macdonald Hall', 'Richardson Hall', 'Ontario Hall', 'Ellis Hall', 'Jeffery Hall', 'Chernoff Hall', 'Watts Hall', 'Chernoff Auditorium', 'Chown Hall', 'Morris Hall', 'McLaughlin Hall', 'Watson Hall', 'Harrison-LeCaine Hall', 'Agnes Queen’s Art Gallery', 'Grant Hall', 'Nicol Hall', 'Jackson Hall', 'Old Medical Building', 'Humphrey Hall', 'Kathleen Ryan Hall', 'Louise D. Acton Building', 'Cancer Research Institute', 'Clark Hall', 'Carruthers Hall', 'Cataraqui Building', 'Abramsky Hall', 'Dupuis Hall', 'Goodwin Hall', 'Walter Light Hall', 'Stirling Hall', 'LaSalle Building', 'Theological Hall', "Central Heating Plant - Queen's University & KGH", 'Kingston Hall', 'Grey House', 'Botterell Hall', 'University Club', 'Etherington Hall', 'Richardson House', 'Kidd House', 'Physical Plant Services Shed', "Queen's School of Medicine", 'Kinesiology Building', 'The Grad Club', "St. James' Anglican Church", 'Summerhill', 'Beamish-Munro

In [10]:
def Interface(start: overpy.Node, terminal: overpy.Node):
  #world = World(all_campus_result,start,terminal=terminal)
  print("Training the Model")
  #agent = qlearning(world, alpha=alpha, gamma=gamma, e=e)
  print("Training done")

  #agent.e

  print("Showing Best route")


  


Interface(s,t)


Training the Model
Training done
Showing Best route


In [11]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd

In [12]:
#Define coordinates of where we want to center our map
campus_coords = [44.2265204, -76.4959298]

#Create the map
my_map = folium.Map(location = campus_coords, zoom_start = 16)

#Create markers
start_coords = [s.lat,s.lon]
terminal_coords = [t.lat,t.lon]

folium.Marker(start_coords, popup = 'Start').add_to(my_map)
folium.Marker(terminal_coords, popup = 'End').add_to(my_map)

#Display the map
my_map

#Whats left to display a route
# Generate a list of way ids in our code
# (do an api call?) to get OSM data for these ways
# convert .osm to .geojson (done, using !osm) !osmtogeojson file1.osm > file2.geojson

In [13]:
#!https://www.overpass-api.de/api/interpreter?data=[out:json];node[highway=speed_camera];out%20meta;
#!https://www.overpass-api.de/api/interpreter?data=[out:json];node[highway=speed_camera](43.46669501043081,-5.708215989569187,43.588927989569186,-5.605835010430813);out%20meta; --output-document=out.xml
!https://www.overpass-api.de/api/interpreter?data=node[highway=speed_camera](43.46669501043081,-5.708215989569187,43.588927989569186,-5.605835010430813);out&20meta;;

#!wget --post-file=query.txt http://overpass-api.de/api/interpreter --output-document=out.xml



/bin/bash: -c: line 0: syntax error near unexpected token `43.46669501043081,-5.708215989569187,43.588927989569186,-5.605835010430813'
/bin/bash: -c: line 0: `https://www.overpass-api.de/api/interpreter?data=node[highway=speed_camera](43.46669501043081,-5.708215989569187,43.588927989569186,-5.605835010430813);out&20meta;;'


In [19]:
#selectedNodeIds=""
#for nodes in 
#overpass.query(node[])
#Not finished but so close -> need to figure out how to put this stupid geojson file into the foliam map and we are set
!cp /content/cache/api-f6f7201ed9c847b8c895891416142873af55d69f /content/api1.osm
!osmtogeojson /content/api1.osm > /content/way.geojson
geojsonFeature = "/content/way.geojson"
#print(geojsonFeature)
folium.GeoJson(geojsonFeature).addTo(my_map);

JSONDecodeError: ignored

In [18]:
!npm --help
!npm search s

node: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by node)
node: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.28' not found (required by node)
